In [2]:
# Load the packages
import pandas as pd
import altair as alt

### 1. Loading the Data

In this step, we load the datasets into Python using the `pandas` library to demonstrate that the dataset can be loaded into Python.
We read the `players.csv` and `sessions.csv` files, which contain essential details about player characteristics and session activities, respectively.

The code used for this step accomplishes the following:
- It imports the data from the relative file paths into pandas DataFrames.
- It displays the first three rows of each dataset to provide a quick overview of the data structure and contents, allowing us to verify that the datasets have been loaded correctly.

In [3]:
# 1. Load the datasets
players = pd.read_csv('data\players.csv')
sessions = pd.read_csv('data\sessions.csv')

# Display the first few rows of each dataset
players.head(3), sessions.head(3)

(  experience  subscribe                                        hashedEmail  \
 0        Pro       True  f6daba428a5e19a3d47574858c13550499be23603422e6...   
 1    Veteran       True  f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...   
 2    Veteran      False  b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...   
 
    played_hours       name gender  age  individualId  organizationName  
 0          30.3     Morgan   Male    9           NaN               NaN  
 1           3.8  Christian   Male   17           NaN               NaN  
 2           0.0      Blake   Male   17           NaN               NaN  ,
                                          hashedEmail        start_time  \
 0  bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...  30/06/2024 18:12   
 1  36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...  17/06/2024 23:33   
 2  f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...  25/07/2024 17:34   
 
            end_time  original_start_time  original_end_time  
 0  30/06/2024 18:24

### 2. Wrangling the Data for Further Detailed Analysis

Data wrangling is a critical step in the data analysis process where we prepare the raw data for meaningful analysis. This stage involves several key tasks that ensure the data is in the appropriate format, is clean, and accurately represents the information we intend to analyze. Proper data wrangling can significantly enhance the quality of the insights derived from the data and make the analysis process smoother and more efficient.

#### Objectives of Data Wrangling:
1. Convert All Data into Proper Data Type

2. Verify the Integrity of the Data

3. Handling Missing Data and Dropping Unnecessary Columns 

4. Combine two dataframe

By meticulously addressing these aspects, we can ensure that our dataset is well-structured, clean, and primed for subsequent analysis stages.


#### 2-1. Convert All Data into Proper Data Type: 

Proper data types are essential for efficient data processing and accurate analysis. This step ensures that each column in our datasets is stored in the most appropriate format, reflecting the nature of the data and optimizing for both memory usage and processing speed. We will convert date columns to datetime objects and other relevant columns to categorical or numerical types based on their content and role in our analysis.


In [5]:
## check the origin data type
print(players.dtypes)
print()
print(sessions.dtypes)

experience           object
subscribe              bool
hashedEmail          object
played_hours        float64
name                 object
gender               object
age                   int64
individualId        float64
organizationName    float64
dtype: object

hashedEmail             object
start_time              object
end_time                object
original_start_time    float64
original_end_time      float64
dtype: object


In [7]:
# Converting data types in 'players'
players['experience'] = players['experience'].astype('category')
players['gender'] = players['gender'].astype('category')

# Converting data types in 'sessions'
sessions['start_time'] = pd.to_datetime(sessions['start_time'],dayfirst=True)
sessions['end_time'] = pd.to_datetime(sessions['end_time'],dayfirst=True)

## check the origin data type again
print(players.dtypes)
print()
print(sessions.dtypes)

experience          category
subscribe               bool
hashedEmail           object
played_hours         float64
name                  object
gender              category
age                    int64
individualId         float64
organizationName     float64
dtype: object

hashedEmail                    object
start_time             datetime64[ns]
end_time               datetime64[ns]
original_start_time           float64
original_end_time             float64
dtype: object


By converting `experience` and `gender` in the `players` dataset to categorical types, we enhance the efficiency of our data storage and simplify the analysis involving these variables. 

By converting `start_time` and `end_time` from the `sessions` dataset to datetime, we enhance accurate and efficient time-based calculations. This adjustment ensures that our data handling is robust and that our analyses will be based on correctly formatted data, enabling precise and reliable results.


#### 2-2. Verifying the Integrity of the Data

Ensuring data integrity is a critical step before having any other data analysis. We need to check for missing values across the datasets. Missing data may significantly impact the process of the analysis, leading to biased or incorrect conclusions if not properly addressed. By identifying missing values before further analysis, we can decide on appropriate strategies for handling them, such as imputation or removal, ensuring a robust dataset for subsequent analyses.

In [8]:
# 2. verifying the integrity of the data

## Check for missing values in each column
missing_data_counts_players = players.isnull().sum()
missing_data_counts_sessions = sessions.isnull().sum()
print(missing_data_counts_players)
print()
print(missing_data_counts_sessions)

experience            0
subscribe             0
hashedEmail           0
played_hours          0
name                  0
gender                0
age                   0
individualId        196
organizationName    196
dtype: int64

hashedEmail            0
start_time             0
end_time               2
original_start_time    0
original_end_time      2
dtype: int64


By checking for missing values in each column of the `players` dataset, we can see that most columns in the `players` dataset are complete except for `individualId` and `organizationName`, which are entirely missing. This indicates that these columns may not provide any useful information for our analysis, as they contain no data at all. So we will drop them in the following wrangling.

By checking for missing values in each column of the `players` dataset, we can see that the `end_time` and `original_end_time` columns each have 2 missing entries, suggesting minor issues with data recording for these specific sessions. Considering the number of missing data is small, we may fill missing `end_time` with the start_time plus the average session duration. The `original_end_time` is not needed for our future analysis and will be drop in following steps, so we can ignore it.


#### 2-3. Handling Missing Data and Dropping Unnecessary Columns

##### Dropping Empty Columns:
In the `players` DataFrame, we identified two completely empty columns: `individualId` and `organizationName`. Since these columns contain no useful data, they were dropped from the DataFrame. Removing such columns is beneficial as it reduces the complexity of the data and focuses our analysis on variables that provide valuable information.

##### Imputing Missing Data:
In the `sessions` DataFrame, we noticed missing values in the `end_time` column. To address this, we calculated the average duration of all sessions and used this value to impute the missing `end_time` for sessions where it was absent. This approach assumes that the missing sessions have an average duration, which helps maintain the integrity of session length calculations across the dataset.

##### Dropping Redundant Time Columns:
We also dropped `original_start_time` and `original_end_time` from the `sessions` DataFrame. These columns were deemed unnecessary for our analysis, possibly because they were redundant or did not align with our specific analytical goals. By removing these, we simplify the dataset further, ensuring that each remaining variable has a clear purpose in our analysis.

##### Verifying Data Integrity Post-Cleanup:
After these modifications, we rechecked for missing values in both datasets to ensure that all issues were appropriately addressed. This final check helps confirm that our dataset is now clean, concise, and ready for deeper analysis, free of missing values and irrelevant columns.

By performing these steps, we enhance the dataset’s usability and ensure our subsequent analyses are based on accurate and relevant data.

In [9]:
# handle missing data, drop needless columns

## drop players 's empty column ['individualId', 'organizationName']
players.drop(columns=['individualId', 'organizationName'],inplace = True)

# Calculate the average duration in seconds (if time is in datetime format)
average_duration = (sessions['end_time'] - sessions['start_time']).mean()

# Iterate over the DataFrame rows using iterrows()
for index, row in sessions.iterrows():
    # Check if 'end_time' is NaN
    if pd.isnull(row['end_time']):
        # Impute missing 'end_time' by adding the average duration to 'start_time'
        sessions.at[index, 'end_time'] = row['start_time'] + average_duration

## drop sessions 's empty column ['individualId', 'organizationName']
sessions.drop(columns=['original_start_time', 'original_end_time'],inplace = True)

## Check for missing values in each column again
missing_data_counts_players = players.isnull().sum()
missing_data_counts_sessions = sessions.isnull().sum()
print(missing_data_counts_players)
print()
print(missing_data_counts_sessions)

experience      0
subscribe       0
hashedEmail     0
played_hours    0
name            0
gender          0
age             0
dtype: int64

hashedEmail    0
start_time     0
end_time       0
dtype: int64


#### 2-4. Combining DataFrames and Analyzing Session Durations

##### Calculating Session Durations:
Before merging, we calculate the duration of each session within the `sessions` dataframe. The duration is computed by subtracting the `start_time` from the `end_time` and converting this difference into hours. This step is essential for understanding how long players engage with the game in each session, which is a critical aspect of player behavior analysis.

##### Aggregating Session Times Per Player:
After calculating individual session durations, we aggregate these durations by player using the `hashedEmail` as a unique identifier. This aggregation provides the total session time spent by each player across all their sessions, giving us a comprehensive view of player engagement.

##### Merging Session Data with Player Profiles:
With the total session times calculated, we merge this data into the `players` dataframe. This merge operation enriches the player profiles with session duration data, allowing us to analyze player characteristics in conjunction with their gaming behavior.

##### Comparing Session Times with Reported Played Hours:
After merging, we compute the discrepancy between the reported `played_hours` in the `players` dataset and the calculated session durations. This discrepancy helps identify potential errors or anomalies in how session times are recorded or reported. It also provides insights into the accuracy of data collection methods used in the game's backend systems.

##### Evaluating Data Integrity:
The final step involves evaluating these discrepancies. By examining the differences between reported and calculated times, we can assess the reliability of our data sources and make informed decisions about further data cleaning or investigation needs.

The code and these steps ensure that our analysis is built on accurate, reliable data, and provide a solid foundation for further exploratory and predictive analysis.

In [10]:
# Calculate session durations in hours
sessions['session_duration'] = (sessions['end_time'] - sessions['start_time']).dt.total_seconds() / 3600

# Aggregate total session time per player
total_session_time = sessions.groupby('hashedEmail')['session_duration'].sum()

# Merge this with player data
players = players.merge(total_session_time, how='left', left_on='hashedEmail', right_index=True)

# Compare the calculated total session time with 'played_hours'
players['discrepancy'] = players['played_hours'] - players['session_duration']

# Check discrepancies
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,session_duration,discrepancy
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,33.650000,-3.350000
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,4.250000,-0.450000
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,0.083333,-0.083333
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,0.833333,-0.133333
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,0.150000,-0.050000
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,0.350000,-0.050000
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,0.083333,-0.083333
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,2.983333,-0.683333
